In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import os

In [0]:
# Python notebook cell to show more output, rather than just showing the last
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# pandas option to show more columns, rather than truncating some in the middle
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', -1)

In [0]:
data = pd.read_csv("cars.csv")
#print(data)
data.head()

,Vehicle Name,Type,Drive,Retail,Dealer,Engine,Cylinders,Horsepower,CityMPG,HighwayMPG,Weight,Wheelbase,Length,Width
0,Acura 3.5 RL,Sedan,Front-wheel,43755,39014,3.5,6,225,18,24,3880,115,197,72
1,Acura 3.5 RL Navigation,Sedan,Front-wheel,46100,41100,3.5,6,225,18,24,3893,115,197,72
2,Acura MDX,SUV,All-wheel,36945,33337,3.5,6,265,17,23,4451,106,189,77
3,Acura NSX S,Sports,Rear-wheel,89765,79978,3.2,6,290,17,24,3153,100,174,71
4,Acura RSX,Sedan,Front-wheel,23820,21761,2.0,4,200,24,31,2778,101,172,68


In [0]:
#data_reduced = data.drop(['Company'], axis=1)    # drop label 'Company' from column (axis=1)
#data_reduced.head(20)

In [0]:
price_weight_horsepower = data[['Retail','Weight','Horsepower']]
price_weight_horsepower.head()

,Retail,Weight,Horsepower
0,43755,3880,225
1,46100,3893,225
2,36945,4451,265
3,89765,3153,290
4,23820,2778,200


In [0]:
# use sklearn StandardScaler to standardize the data to mean 0, standard deviation 1,  Z=(X-u)/s
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [0]:
scaled_price_weight_horsepower = scaler.fit_transform(price_weight_horsepower)
print(price_weight_horsepower.head(10))
for i in range(10):
  print('{0:2d}-->'.format(i),end="")
  print('price:{0:9.6f}  '.format(scaled_price_weight_horsepower[i][0]),end="")
  print('weight:{0:9.6f}  '.format(scaled_price_weight_horsepower[i][1]),end="")
  print('horsepower:{0:9.6f}  '.format(scaled_price_weight_horsepower[i][2]),end="")
  print('')

   Retail  Weight  Horsepower
0  43755   3880    225       
1  46100   3893    225       
2  36945   4451    265       
3  89765   3153    290       
4  23820   2778    200       
5  33195   3575    270       
6  26990   3230    200       
7  25940   3252    170       
8  35940   3638    170       
9  42490   3814    220       
 0-->price: 0.534227  weight: 0.492905  horsepower: 0.150424  
 1-->price: 0.653268  weight: 0.511342  horsepower: 0.150424  
 2-->price: 0.188527  weight: 1.302729  horsepower: 0.720452  
 3-->price: 2.869863  weight:-0.538168  horsepower: 1.076720  
 4-->price:-0.477746  weight:-1.070014  horsepower:-0.205843  
 5-->price:-0.001837  weight: 0.060336  horsepower: 0.791706  
 6-->price:-0.316825  weight:-0.428962  horsepower:-0.205843  
 7-->price:-0.370127  weight:-0.397761  horsepower:-0.633365  
 8-->price: 0.137510  weight: 0.149687  horsepower:-0.633365  
 9-->price: 0.470011  weight: 0.399300  horsepower: 0.079171  
